In [0]:
from pyspark.sql.functions import col, split, regexp_replace

# Read from Bronze table created in notebook 1
df = spark.table("nlp_databricks.default.bronze_fasttext_job")

# Split "value" into label + review_text
parts = split(col("value"), " ", 2)

df_silver = (
    df
    .withColumn("label_raw", parts.getItem(0))           # "__label__2"
    .withColumn("review_text", parts.getItem(1))         # rest of the line
    .withColumn("label", regexp_replace("label_raw", "__label__", ""))  # "2"
    .drop("value", "label_raw")
)

df_silver.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("nlp_databricks.default.silver_fasttext_job")
